In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

import json
import pickle

import numpy as np
import pandas as pd


def one_hot_encode(pca,df_original, column):
    # Definir tamaño de los lotes
    batch_size = 10000  # Ajusta según el tamaño que la RAM permita

    # Inicializar lista para almacenar los resultados parciales
    results = []

    # Procesar el DataFrame en lotes
    for i in range(0, len(df_original), batch_size):
        # Crear lote
        df_batch = df_original.iloc[i:i + batch_size].copy()
        
        # Codificación one-hot para el lote
        df_batch = pd.get_dummies(df_batch, columns=[column])
        
        # Aplicar PCA transform en el lote
        result_batch = pca.transform(df_batch)
        
        # Guardar el resultado en la lista
        results.append(result_batch)

    # Concatenar todos los resultados
    result_full = np.vstack(results)
    return result_full

In [2]:

def PCA_one_hot(df,column, n_components,th=0.9):
    # One hot encoding

    df_original = df.copy()

    categories = df[column].cat.categories

    ind_random = np.random.permutation(100000)
    df = pd.DataFrame({column:df_original[column].iloc[ind_random]})

    df[column] = df[column].astype('category')
    df[column] = df[column].cat.set_categories(categories)
    df_one_hot = pd.get_dummies(df, columns=[column])

    # PCA
    pca = PCA(n_components=n_components)
    pca.fit(df_one_hot)
        

    er_cum = np.cumsum(pca.explained_variance_ratio_)
    # find pca.explained_variance_ratio_ > 0.05
    idx = np.where(er_cum > th)
    if  len(idx[0]) > 0:
        idx = idx[0][0]
    else:
        idx = n_components-1
    
    ncateg = df[column].cat.categories.shape[0]

    print(f"n_components = {idx+1} | n_categories = {ncateg}")

    n_components = idx+1


    df_full = pd.DataFrame({column:df_original[column]})
    result_full = one_hot_encode(pca,df_full, column)

    result_full = result_full[:, :n_components]


    df_new = pd.DataFrame(result_full, columns=[column+f"PC{i+1}" for i in range(n_components)])
    df_new.index = df_full.index

    for col in df_new.columns:
        df_original[col] = df_new[col]
    
    df_original.pop(column)

    fig = plt.figure()
    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
    plt.xlabel("Principal component")
    plt.title("Cumulative explained variance ratio | n_components = "+str(n_components) + " | n_categories = "+str(ncateg) + " | columns = "+column)
    # horizontal line
    plt.axhline(y=th, color='r', linestyle='-')
    # vertical line
    plt.axvline(x=idx, color='r', linestyle='-')
    
    
    return df_original,pca, n_components
# 

In [3]:

def PCA_one_hot_apl(pca,df,column, n_components):
    # One hot encoding

    df_original = df.copy()

    df_full = pd.DataFrame({column:df_original[column]})
    result_full = one_hot_encode(pca,df_full, column)

    result_full = result_full[:, :n_components]


    df_new = pd.DataFrame(result_full, columns=[column+f"PC{i+1}" for i in range(n_components)])
    df_new.index = df_full.index

    for col in df_new.columns:
        df_original[col] = df_new[col]
    
    df_original.pop(column)

    
    return df_original
# 

In [4]:
def predict(df_transactions_full, metadata):
    pd.options.mode.chained_assignment = None  # default='warn'
    amount_mean = metadata['amount_mean']
    amount_std = metadata['amount_std']
    full_categories = metadata['full_categories']
    pca_list = metadata['pca_list']
    n_components_list = metadata['n_components_list']

    df_transactions_full = df_transactions_full.copy()
    
    df_transactions_full["amount"] = df_transactions_full["amount"].apply(lambda x: x.replace("$", "")).astype(float)
    df_transactions_full['amount'] = (df_transactions_full['amount'] - amount_mean) / amount_std

    # date -> day, month, year and remove date
    df_transactions_full['date'] = pd.to_datetime(df_transactions_full['date'])
    df_transactions_full['day'] = df_transactions_full['date'].dt.day
    df_transactions_full['month'] = df_transactions_full['date'].dt.month
    df_transactions_full["hour"] = df_transactions_full["date"].dt.hour
    df_transactions_full["weekday"] = df_transactions_full["date"].dt.weekday
    df_transactions_full.pop("date")

    df_transactions_full["use_chip"] = df_transactions_full["use_chip"].astype('category')
    df_transactions_full["use_chip"] = df_transactions_full["use_chip"].cat.set_categories(metadata["use_chip_categories"])

    iter = 0
    for col in ["merchant_id", "merchant_city", "merchant_state", "mcc"]:
        df_transactions_full[col] = df_transactions_full[col].astype('category')
        # current caterogies
        df_transactions_full[col] = df_transactions_full[col].cat.set_categories(full_categories[iter])

        # Nan -> others
        iter += 1

    # id and client_id are not useful
    id_trans = df_transactions_full.pop("id")
    df_transactions_full.pop("client_id")
    df_transactions_full.pop("zip")
    df_transactions_full.pop("errors")


    df_transactions_full["merchant_city"][df_transactions_full["merchant_city"].isna()] = "others"
    df_transactions_full["merchant_state"][df_transactions_full["merchant_state"].isna()] = "others"
    df_transactions_full["mcc"][df_transactions_full["mcc"].isna()] = "others"

    # Transformar los datos

    iter = 0
    for col in ["merchant_id", "merchant_city", "mcc","merchant_state"]:
        pca = pca_list[iter]
        n_components = n_components_list[iter]
        df_transactions_full =     PCA_one_hot_apl(pca,df_transactions_full,col, n_components)
        iter += 1

    df_transactions_full.pop("card_id")

    df_transactions_full = pd.get_dummies(df_transactions_full, columns=["use_chip"])

    model = metadata['model']
    scaler = metadata['scaler']

    X = df_transactions_full.drop(columns=['is_fraud'])
    X_test_scaled = scaler.transform(X)
    y_pred = model.predict(X_test_scaled)

    pred = pd.DataFrame({
        "id": id_trans
        ,"is_fraud": y_pred
    })

    return pred

In [5]:
metadata = pickle.load(open('metadata.pkl', 'rb'))

amount_mean = metadata['amount_mean']
amount_std = metadata['amount_std']
full_categories = metadata['full_categories']
pca_list = metadata['pca_list']
n_components_list = metadata['n_components_list']

data_prediction = json.load(open('../old/predictions_3.json'))

df_transactions_full = pd.read_csv('../data/transactions_data.csv')

df_transactions_full['is_fraud'] = df_transactions_full['id'].apply(lambda x: data_prediction["target"][str(x)] 
                                                                    if str(x) in data_prediction["target"] 
                                                                    else "unknown")
df_transactions_full = df_transactions_full[df_transactions_full['is_fraud'] != "unknown"]
df_transactions_full['is_fraud'] = df_transactions_full['is_fraud'].apply(lambda x: 1 if x == "Yes" else 0)



In [6]:
# warning message off 
pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
nrows = df_transactions_full.shape[0]
arange = np.linspace(0,nrows,1000).astype(int)
pred_list = []
for i in range(len(arange)-1):
    print(f"Processing {i}/{len(arange)}")
    pred = predict(df_transactions_full.iloc[arange[i]:arange[i+1]], metadata)
    pred_list.append(pred)



Processing 0/1000
Processing 1/1000
Processing 2/1000


/tmp/ipykernel_18681/3200478325.py:41: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_transactions_full["merchant_city"][df_transactions_full["merchant_city"].isna()] = "others"
/tmp/ipykernel_18681/3200478325.py:42: FutureWarning: Chained

Processing 3/1000
Processing 4/1000
Processing 5/1000


/tmp/ipykernel_18681/3200478325.py:41: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_transactions_full["merchant_city"][df_transactions_full["merchant_city"].isna()] = "others"
/tmp/ipykernel_18681/3200478325.py:42: FutureWarning: Chained

Processing 6/1000
Processing 7/1000
Processing 8/1000


...

Processing 991/1000
Processing 992/1000
Processing 993/1000


/tmp/ipykernel_18681/3200478325.py:41: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_transactions_full["merchant_city"][df_transactions_full["merchant_city"].isna()] = "others"
/tmp/ipykernel_18681/3200478325.py:42: FutureWarning: Chained

Processing 994/1000
Processing 995/1000
Processing 996/1000


/tmp/ipykernel_18681/3200478325.py:41: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_transactions_full["merchant_city"][df_transactions_full["merchant_city"].isna()] = "others"
/tmp/ipykernel_18681/3200478325.py:42: FutureWarning: Chained

Processing 997/1000
Processing 998/1000


In [8]:
pred_list

[            id  is_fraud
 3      7475331         0
 8      7475336         0
 9      7475337         0
 15     7475343         0
 34     7475364         0
 ...        ...       ...
 13315  7491283         0
 13320  7491289         0
 13321  7491290         0
 13329  7491302         0
 13336  7491312         0
 
 [2893 rows x 2 columns],
             id  is_fraud
 13339  7491314         0
 13341  7491319         0
 13342  7491320         0
 13348  7491328         0
 13352  7491333         0
 ...        ...       ...
 27389  7508112         0
 27394  7508118         0
 27399  7508124         0
 27404  7508129         0
 27411  7508139         0
 
 [2894 rows x 2 columns],
             id  is_fraud
 27417  7508147         0
 27426  7508157         0
 27427  7508158         0
 27431  7508163         0
 27438  7508170         0
 ...        ...       ...
 40419  7523715         0
 40432  7523732         0
 40433  7523733         0
 40435  7523735         0
 40437  7523738         0
 
 [2894

In [9]:
pred_concat = pd.concat(pred_list)

In [10]:
pred_concat["is_fraud"].value_counts()

is_fraud
0    2861759
1      29193
Name: count, dtype: int64

{
    "target": {
        "19150685": "Yes/No",
        "10892174": "Yes/No",
        "16354005": "Yes/No",
        "9887800": "Yes/No",
        "8233352": "Yes/No",
        "12865486": "Yes/No",

In [11]:
r_dict = {
    "target": {}
}

for i in range(pred_concat.shape[0]):
    r_dict["target"][str(pred_concat["id"].iloc[i])] = "Yes" if pred_concat["is_fraud"].iloc[i] == 1 else "No"

In [12]:
save_in = "../predictions/predictions_3.json"

In [13]:
import json

with open(save_in, 'w') as fp:
    json.dump(r_dict, fp, indent=4)